## This class will hold the database creation and population work

#### Install sqlalchemy_utils if they aren't already installed:

In [9]:
import importlib.util
import sys

name = 'sqlalchemy_utils'

if name in sys.modules:
    print(f"{name!r} already in sys.modules")
elif (spec := importlib.util.find_spec(name)) is not None:
    module = importlib.util.module_from_spec(spec)
    sys.modules[name] = module
    spec.loader.exec_module(module)
    print(f"{name!r} has been imported")
else:
    print(f"can't find the {name!r} module")


#If this code doesn't work as intended, just delete everything except the following string and run it: pip install sqlalchemy_utils

'sqlalchemy_utils' already in sys.modules


#### Create the database:

In [17]:
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine, text
from sqlalchemy_utils import database_exists, create_database

engine = create_engine('sqlite:///it_ticketing_system.db', echo=True)
if not database_exists(engine.url):
    create_database(engine.url)

if sqlalchemy.__version__ < '2.0.0':
    raise ValueError('Please upgrade your version of SQLAlchemy to 2.0.0 or greater')


### Using sqlite3 to alter/drop tables if they're already created.

In [36]:
import sqlite3
db_file = 'it_ticketing_system.db'
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()
cur.execute("DROP TABLE IF EXISTS fact_tickets")
cur.execute("DROP TABLE IF EXISTS dim_departments")
cur.execute("DROP TABLE IF EXISTS dim_organizations")
cur.execute("DROP TABLE IF EXISTS fact_ticket_lines")
cur.execute("DROP TABLE IF EXISTS dim_technicians")
cur.execute("DROP TABLE IF EXISTS dim_users")
cnn.close()

#### Create the tables for the database:

In [ ]:
from typing import List, Optional 
from sqlalchemy import Table, Column, String, DateTime, Float, Integer, ForeignKey, MetaData
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import Session, relationship
from sqlalchemy import select

meta = MetaData()

fact_tickets = Table(
    'fact_tickets'
    , meta
    , Column('ticket_id', Integer, primary_key=True)
    , Column('user_id', Integer, ForeignKey('dim_users.user_id'))
    , Column('department_id', Integer, ForeignKey('dim_departments.department_id'))
    , Column('prior_ticket_id', Integer)
    , Column('ticket_category', String, nullable=False)
    , Column('open_date_time', DateTime, nullable=False)
    , Column('close_date_time', DateTime)
    , Column('status', String)
    , Column('description', String, nullable=False)
    , Column('subject', String, nullable=False)
)

fact_ticket_lines = Table(
    'fact_ticket_lines'
    , meta
    , Column('ticket_line_id', Integer, primary_key=True)
    , Column('ticket_id', Integer, ForeignKey('fact_tickets.ticket_id'))
    , Column('technician_id', Integer, ForeignKey('dim_users.user_id'))
    , Column('assignment_date_time', DateTime, nullable=False)
    , Column('completion_date_time', DateTime)
    , Column('notes', String)
)

dim_technicians = Table(
    'dim_technicians'
    , meta
    , Column('technician_id', Integer, primary_key=True)
    , Column('user_id', Integer, ForeignKey('dim_users.user_id'))
    , Column('manager_id', Integer)
)

dim_users = Table(
    'dim_users'
    , meta
    , Column('user_id', Integer, primary_key=True)
    , Column('organization_id', Integer, ForeignKey('dim_organizations.organization_id'))
    , Column('department_id', Integer, ForeignKey('dim_departments.department_id'))
    , Column('last_name', String, nullable=False)
    , Column('first_name', String, nullable=False)
    , Column('phone_number', String)
    , Column('email_address', String, nullable=False)
    , Column('title', String)
)

dim_departments = Table(
    'dim_departments'
    , meta
    , Column('department_id', Integer, primary_key=True)
    , Column('organization_id', Integer, ForeignKey('dim_organizations.organization_id'))
    , Column('name', String, nullable=False)
    , Column('phone_number', String)
    , Column('email_address', String, nullable=False)
)

dim_organizations = Table(
    'dim_organizations'
    , meta
    , Column('organization_id', Integer, primary_key=True)
    , Column('name', String, nullable=False)
    , Column('phone_number', String)
    , Column('email_address', String, nullable=False)
    , Column('state', String, nullable=False)
    , Column('city', String, nullable=False)
    , Column('zip_code', Integer, nullable=False)
    , Column('street_address', String, nullable=False)
)


meta.create_all(engine)

#### Initial Database Definitions:

In [ ]:
db_file = 'it_ticketing_system.db'
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()
tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

def bool_to_yes_no(value):
    if value == 0:
        return 'NO'
    elif value == 1:
        return 'YES'

for table in tables:
    print('------------------')
    print('Table Name: ' + table[0])
    print('------------------')
    columns = cur.execute(f'PRAGMA table_info(\'%s\');' % table[0]).fetchall()
    for column in columns:
        col_id, col_name, col_type, col_notnull, col_default, col_pk = column
        print(f"  Column: {col_name}")
        print(f"    Type: {col_type}")
        print(f"    Not Null: {bool_to_yes_no(col_notnull)}")
        print(f"    Default Value: {col_default}")
        print(f"    Primary Key: {bool_to_yes_no(col_pk)}")

cnn.close()